# Miscellaneous GSL utilities

Some isolated utilities, which sometimes prefigure features to come in the next versions of the C ++ standard and its library.

## "C-style" strings `gsl::zstring` and `gsl::czstring`

The two aliases below simply allow to draw attention to strings that are supposed to end with a null character:
* `zstring`: `char *`, being `nullptr` or pointing to a C-style string;
* `czstring`: `char const *`, being `nullptr` or pointing to a C-style string.

If your C-style string pointer should not be null, use `gsl::not_null<zstring>`.

If your string is not supposed to end with a null character, use `string_view`.

## For generic and efficient access to strings: `std::string_view`

The `std::string` class is both very convenient and very expensive.

On the model of `gsl::span`, a *view* was proposed for string, renamed into `std::string_view` upon integration into C++17.

Composed of a pointer and a size, the `std::string_view` class allows to create easily and quickly a **constant non-owner view to a sequence of contiguous characters in memory**: not just on top of a `std::string`, but also a simple literal string or a `Qstring` (Qt).

As `string_view` stores the size of the string, the underlying string does not necessarily need to end with a character `0`.

Writing a function receiving a `string_view` (rather than a `std::string const &`) has benefits:

* more efficiency than `std::string` with a literal string;

* capacity of receiving all kinds of strings as input;

* efficient creation of a substrings because the final `0` character is no longer necessary.

**Be careful**: the `string_view` does not own or duplicate the underlying data. Be careful not to use a view when its underlying data has disappeared.

In addition, by going through a `string_view`, we lose the guarantee of having the final `0`. If the string must then be passed to other functions that expect the terminal `0`, it is better to use `string` from the beginning.

## Contracts

GSL offers two dedicated assertions to validate *pre-conditions* and *post-conditions*.
* `Expects(p)`: stop the application unless `p == true`.
* `Ensures(p)`: stop the application unless `p == true`.

These assertions are **currently implemented via macros**, and must be placed in the body of functions, while waiting for future decisions of the standardization committee that deals with contracts and syntax of assertions.

In the [Contract Proposal](http://www.open-std.org/jtc1/sc22/wg21/docs/papers/2016/p0380r1.pdf), it is proposed to to move these declarations to the level of function declarations and use some specifiers such as `[[expects: p]]`.

The contracts are not yet in C++23. Perhaps in C++ 26 ?

In [ ]:
%%file tmp.gsl-utilities.cpp

#include <iostream>
#include <gsl/gsl>

void divide( double num, double den ) {
    Expects(den!=0) ;
    std::cout<<(num/den)<<std::endl ;
}

int main() {
    divide(5,0) ;
}

In [ ]:
!rm -f tmp.gsl-utilities.exe && g++ -std=c++17 -I./ tmp.gsl-utilities.cpp -o tmp.gsl-utilities.exe

In [ ]:
!./tmp.gsl-utilities.exe

## Final action: `finally()`

As a last resort, when the resources management tools are not sufficient, we can define a function to be invoked at the end of a block.

In [ ]:
%%file tmp.gsl-utilities.cpp

#include <iostream>
#include <gsl/gsl>

struct Demo {
    Demo() { std::cout<<"Constructor"<<std::endl ; }
    ~Demo() { std::cout<<"Destructor"<<std::endl ; }
} ;

int main() {
    Demo * d {new Demo} ;
    auto _ { gsl::finally( [d](){ delete d ; } ) } ;
    // ...
}

In [ ]:
!rm -f tmp.gsl-utilities.exe && g++ -std=c++17 -I./ tmp.gsl-utilities.cpp -o tmp.gsl-utilities.exe

In [ ]:
!./tmp.gsl-utilities.exe

## Numerical utilities: `narrow_cast<T>(x)` and `narrow<T>(x)`

The first one, `narrow_cast<T>(x)`, is just a statement for programmers or testing tools. It clarifies the fact that the developer wants **voluntarily** to force a value to a less precise type. 

The second one, `narrow<T>(x)`, also checks at runtime that the value `x` was not modified when its type was transformed into a `T`, and throws an exception (or terminates the program) otherwise (if `static_cast<T>(x) != x`).

In [ ]:
%%file tmp.gsl-utilities.cpp

#include <iostream>
#include <gsl/gsl>

int main() {
    double d { 3.14 } ;
    int i { gsl::narrow<int>(d) } ;
    std::cout<<i<<std::endl ;
}

In [ ]:
!rm -f tmp.gsl-utilities.exe && g++ -std=c++17 -I./ tmp.gsl-utilities.cpp -o tmp.gsl-utilities.exe

In [ ]:
!./tmp.gsl-utilities.exe

# Questions ?

## Exercise

Create your own minimal `my_narrow`, so that the program below accept the narrowing of `42`, but will crash when trying to narrow `3.14`.

In [ ]:
%%file tmp.gsl-utilities.cpp

#include <iostream>
#include <cassert>

// ... PUT HERE YOUR IMPLEMENTATION OF my_narrow  ...
 
int main()
 {
  double d1 {42} ;
  int i1 {my_narrow<int>(d1)} ;
  std::cout<<i1<<std::endl ;
    
  double d2 {3.14} ;
  int i2 {my_narrow<int>(d2)} ;
  std::cout<<i2<<std::endl ;
 }

In [ ]:
!rm -f tmp.gsl-utilities.exe && g++ -std=c++17 -I./ tmp.gsl-utilities.cpp -o tmp.gsl-utilities.exe

In [ ]:
!./tmp.gsl-utilities.exe

## Sources

* https://stackoverflow.com/questions/40127965/how-exactly-is-stdstring-view-faster-than-const-stdstring
* http://isocpp.github.io/CppCoreGuidelines/CppCoreGuidelines#gsl-guidelines-support-library
* http://modernescpp.com/index.php/c-core-guideline-the-guidelines-support-library
* http://nullptr.nl/2018/08/refurbish-legacy-code/

© *CNRS 2024*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*